In [1]:
from Alice_Macros import *
from sklearn.cluster import KMeans
from statsmodels.stats.multitest import multipletests
from scipy.stats.distributions import hypergeom, norm
import pickle, sys, ete3, tqdm
from scipy import sparse as sp
from TreeOperations import PhyFisher
import tqdm
from TreeOperations import GetPhyloContingency
import matplotlib.pylab as plt
from rpy2 import robjects

In [2]:
def GenerateTree(ntax, birth_rate, death_rate, SaveDir = None):
    t = ete3.Tree(robjects.r("ape::write.tree(ape::rphylo({},{},{},fossils=FALSE))".format(ntax,birth_rate,death_rate))[0])
    for i, node in enumerate(t.traverse()): node.name = str(i)
    if SaveDir:
        if not os.path.exists(SaveDir): os.mkdir(SaveDir)
        pickle.dump(t, open(os.path.join(SaveDir, "tree"), 'wb'))
    return t

In [3]:
def events_to_contingency(events_per_mut):
    tables = []
    row_it = iter(events_per_mut)
    phen = set(next(row_it))
    for gene in tqdm.tqdm(row_it, total=len(events_per_mut)-1, desc='converting events to tables'):
        gene = set(gene)
        tables.append([population - len(set.union(phen, gene)),
                       len(set.difference(phen, gene)),
                       len(set.difference(gene, phen)),
                       len(set.intersection(gene, phen))])
    return tables

In [4]:
node_to_arr = lambda n: np.array(n.todense().astype(np.int8))[0]
def RandomAtributes(tree, population, n_genes, events_per_mut, lims=None):
    # set binary atributes across phylogeny
    order = tree.traverse('preorder')
    root = next(order)
    root.genotype = sp.csr_matrix(np.random.randint(2, size=n_genes))
    for node in tqdm.tqdm(order, total=population, desc='propageting events on tree'):
        if lims is None:
            mut_here = np.any(node.id == events_per_mut, axis=1)
        else:
            mut_here = np.zeros(len(events_per_mut))
            idx_mut = np.argwhere(node.id == events_per_mut)
            idx_mut = idx_mut[idx_mut.T[1]<lims[idx_mut.T[0]]].T[0]
            mut_here[idx_mut] = 1
        node.genotype = sp.csr_matrix(np.abs(node_to_arr(node.up.genotype) - mut_here))
    return tree

In [5]:
def MarginalMaxLiklihood(X, Tree, tip_row_dict, do_tips=False, naming=False, m_rate=0.5):
    # 2 represents {0,1} set
    sp_to_arr = lambda sp_arr: np.array(sp_arr.todense().astype(np.int8))[0]
    wrap = lambda x: sp_to_arr(X[tip_row_dict[x.name]]) if x.is_leaf() and not do_tips else sp_to_arr(x.genotype)
    tree_len = 0
    for _ in Tree.traverse(): tree_len += 1
    for i, node in tqdm.tqdm(enumerate(Tree.traverse('postorder')), total=tree_len, desc='Ancestral Reconstruction: 1st pass'):
        if node.is_leaf():
            if not do_tips:
                node.genotype = X[tip_row_dict[node.name]]
            node.zero_prob = sp.csr_matrix(node.genotype == 0)
            node.one_prob = sp.csr_matrix(node.genotype == 1)
            continue
        if naming: node.name = i
        node.one_prob = ((node.children[0].zero_prob * m_rate) + (node.children[0].one_prob * (1-m_rate))) + ((node.children[1].zero_prob * m_rate) + (node.children[1].one_prob * (1-m_rate)))
        node.zero_prob= ((node.children[0].zero_prob * (1-m_rate)) + (node.children[0].one_prob * m_rate)) + ((node.children[1].zero_prob * (1-m_rate)) + (node.children[1].one_prob * m_rate))
        root.genotype = (root.zero_prob < root.one_prob).astype(int)

In [6]:
def JointMaxLiklihood(X, Tree, tip_row_dict, do_tips=False, naming=False, m_rate=0.5):
    # As explained in FastML
    # 2 represents {0,1} set
    sp_to_arr = lambda sp_arr: np.array(sp_arr.todense().astype(np.int8))[0]
    wrap = lambda x: sp_to_arr(X[tip_row_dict[x.name]]) if x.is_leaf() and not do_tips else sp_to_arr(x.genotype)
    tree_len = 0
    for _ in Tree.traverse(): tree_len += 1
    for node in tqdm.tqdm(Tree.traverse('postorder'), total=tree_len, desc='Ancestral Reconstruction: 1st pass'):
        if node.is_leaf():
            """
            For each OTU y perform the following:
            1a. Let j be the amino acid at y. Set, for each amino acid i: Cy(i)= j. 
                This implies that no matter what is the amino acid in the father of y, j is assigned to node y.
            1b. Set for each amino acid i: Ly(i) = Pij(ty), where ty is the branch length between y and its father.
            """
            if not do_tips:
                node.genotype = X[tip_row_dict[node.name]]
            node.C_0 = node.genotype
            node.C_1 = node.genotype
            node.L_0 = np.ones((node.genotype).shape[1]) * (1 - m_rate)
            node.L_0[node.genotype.indices] = m_rate           
            node.L_1 = np.ones((node.genotype).shape[1]) * m_rate
            node.L_1[node.genotype.indices] = (1-m_rate)
            
            # log
            node.L_1 = np.log(node.L_1)
            node.L_0 = np.log(node.L_0)
            continue
        
        """
        Visit a nonroot internal node, z, which has not been visited yet, 
        but both of whose sons, nodes x and y, have already been visited, 
        i.e., Lx(j), Cx(j), Ly(j), and Cy(j) have already been defined for each j. 
        Let tz be the length of the branch connecting node z and its father. 
        For each amino acid i, compute Lz(i) and Cz(i) according to the following formulae:

        2a. Lz(i) = maxj Pij(tz)× Lx(j) × Ly(j).
        2b. Cz(i) = the value of j attaining the above maximum.
        """
        #s0 = node.children[0].L_0.multiply(node.children[1].L_0)
        #s0 = node.children[0].L_0 * node.children[1].L_0
        s0 = node.children[0].L_0 + node.children[1].L_0
        s1 = node.children[0].L_1 + node.children[1].L_1
        
        # L0_0 = (1-m_rate) * s0
        # L0_1 = m_rate * s1
        L0_0 = np.log(1-m_rate) + s0
        L0_1 = np.log(m_rate) + s1
        # node.L_0 = L0_0.maximum(L0_1) # should be ever expanding right?
        node.L_0 = np.maximum(L0_0, L0_1)
        node.C_0 = L0_0 < L0_1
        
        #L1_0 = m_rate * s0
        #L1_1 = (1-m_rate) * s1
        L1_0 = np.log(m_rate) + s0
        L1_1 = np.log(1-m_rate) + s1
        #node.L_1 = L1_0.maximum(L1_1)
        node.L_1 = np.maximum(L1_0, L1_1)
        node.C_1 = L1_0 < L1_1
        
       
        del node.children[0].L_0
        del node.children[1].L_0
        del node.children[0].L_1
        del node.children[1].L_1
       
    #return Tree
    
    post = Tree.traverse('preorder')
    root = next(post)
    """
    Denote the sons of the root by x, y. 
    For each amino acid k, compute the expression Pk × Lx(k) × Ly(k). 
    Reconstruct r by choosing the amino acid k maximizing this expression. 
    The maximum value found is the likelihood of the best reconstruction.
    """
    root.genotype = sp.csr_matrix(root.L_0 < node.L_1)
    if naming: root.name = 0
    for i,node in tqdm.tqdm(enumerate(post), total=tree_len, desc='Ancestral Reconstruction: 2nd pass'):
        """
        Traverse the tree from the root in the direction of the OTUs, 
        assigning to each node its most likely ancestral character as follows:
        
        5a. Visit an unreconstructed internal node x whose father y has already been reconstructed. 
            Denote by i the reconstructed amino acid at node y.
        5b. Reconstruct node x by choosing Cx(i).
        5c. Return to step 5a until all internal nodes have already been reconstructed.
        """
        if not node.is_leaf(): 
            if naming: node.name = i + 1
            node.genotype = sp.csr_matrix(node.C_0)
            node.genotype[0, node.up.genotype.indices] = node.C_1[node.up.genotype.indices]
            # np.where(node.up.genotype, node.C_1, node.C_0)
        del node.C_0,node.C_1
    
    return Tree 

In [7]:
def MaxParsimonyNoTable(X, Tree, tip_row_dict, do_tips=False, naming=False):
    """
    Reconstruct ancestral characters using MaxParsimony / occam's razor logic...
    Note this implementation relays on binary characters

    The first pass extracts info from the leaves upwards with the following function:
    Intersect when an intersection is available, otherwise yeild union.
    e.g given dijointed sets A,B

         AUB   |      A     |      B
         /^\   |     /^\    |     /^\
        A  B   |    A  AUB  |    B  B

    The second pass brings stable information from root to leaves.
    The root might need randomization to deside on a character.
    We consider events influenced by randomization as risky, and they are documented in 'random'.
    A parent will always be a single character, and a son might get updated by intersection with parent node:

         A     >      A     |     A     >      A     |
        /^\    >     /^\    |    /^\    >     /^\    |
       A  AUB  >    A  A    |   A  B    >    A   B    |

    :param X: sparse csr_mat data to reconstruct
    :param Tree: ete3 tree
    :param tip_row_dict: dict mapping leaf names to row number in X
    :return: A tree where each node has a
            .genotype containing the reconstruction by X
            .random documenting non-determinant values
    """
    # 2 represents {0,1} set
    sp_to_arr = lambda sp_arr: np.array(sp_arr.todense().astype(np.int8))[0]
    wrap = lambda x: sp_to_arr(X[tip_row_dict[x.name]]) if x.is_leaf() and not do_tips else sp_to_arr(x.genotype)
    tree_len = 0
    for _ in Tree.traverse(): tree_len += 1
    for i, node in tqdm.tqdm(enumerate(Tree.traverse('postorder')), total=tree_len, desc='Ancestral Reconstruction: 1st pass'):
        if node.is_leaf():
            if not do_tips:
                node.genotype = X[tip_row_dict[node.name]]
            continue
        if naming: node.name = i
        children = [wrap(c) for c in node.children]
        res = children[0].copy()
        eq = np.equal(*children)
        res[children[0] == 2] = children[1][children[0] == 2]  # 2 is the union {0,1}
        res[children[1] == 2] = children[0][children[1] == 2]
        res[(children[0] != 2) & (children[1] != 2) & ~eq] = 2
        node.genotype = sp.csr_matrix(res)

    post = Tree.traverse('preorder')
    root = next(post)
    root.random = (wrap(root) == 2)
    root.genotype[root.genotype == 2] = np.random.choice([1, 0], size=(root.genotype == 2).sum())
    for node in tqdm.tqdm(post, total=tree_len - 1, desc='Ancestral Reconstruction: 2nd pass'):
        if node.is_leaf(): continue
        parent_ = wrap(node.up)
        node_ = wrap(node)
        res = node_.copy()
        res[node_ == 2] = parent_[node_ == 2]
        node.random = (node.up.random) & (node_ == 2)  # these are unstable positions - will not be counted
        node.genotype = sp.csr_matrix(res)

    return Tree

In [8]:
def GetPhyloTipsContingency(tree, phenotree, phen_ind, skip=0):
    """
    Collects co-occuring events along the phylogeny
    
                                 /--(1,1)
                       /--(0,1)-|
             /--(0,1)-|          \--(0,1)
    --(0,1)-|          \--(1,1)
             \--(0,1)

    Would translate to the following contingency table:

    T\Q| 0  1
    ---------
     0 | 4  1
     1 | 1  0

    :param tree: ete3 tree
    :param phen_ind: index of target T
    :param skip: number of qualities to skip
    :return: A contingency table for each available quality in a nodes 'genotype' value
    # TODO someday, if there are too much kmers build upo ad 3d hist, adding binary 3d objects one node at a time.
    """

    hist_kmers = []
    node_to_arr = lambda n: np.array(n.genotype.todense().astype(np.int8))[0]
    for leaf, phen_leaf in tqdm.tqdm(zip(tree, phenotree), 
                                         total=len(tree), desc='Iterating leaves'):
        
        gene_state = node_to_arr(leaf)[skip:]
        phen_state = np.int8(phen_leaf.genotype[0, phen_ind])
        
        hist = 2 * phen_state + gene_state  # current state
        hist_kmers.append(hist)
            
    bins = lambda x: np.bincount(x[x >= 0], minlength=4)
    contingencies = np.apply_along_axis(bins, 0, hist_kmers)
    return contingencies

In [9]:
def GetPhyloSebsequentScore(tree, phenotree, phen_ind, skip=0, with_rand=False, dist_only=False, dist=None):
    """
    Collects co-occuring events along the phylogeny
    
                                 /--(1,1)
                       /--(0,1)-|
             /--(0,1)-|          \--(0,1)
    --(0,1)-|          \--(1,1)
             \--(0,1)

    Would translate to the following contingency table:

    T\Q| 0  1
    ---------
     0 | 4  1
     1 | 1  0

    :param tree: ete3 tree
    :param phen_ind: index of target T
    :param skip: number of qualities to skip
    :return: A contingency table for each available quality in a nodes 'genotype' value
    """

    subscore = np.zeros(tree.genotype.shape[1] - skip)
    node_to_arr = lambda n: np.array(n.genotype.todense().astype(np.int8))[0]
    for i, (cur_node, phen_node) in tqdm.tqdm(enumerate(zip(tree.traverse(), phenotree.traverse())), 
                                         total=population+1, desc='Iterating tree'):        
        if not cur_node.is_root():
            if not cur_node.is_leaf() and with_rand and cur_node.random[phen_ind]: continue
            node = node_to_arr(cur_node)
            prev_node = node_to_arr(cur_node.up)
            
            gene_state = node[skip:]
            prev_gene_state = prev_node[skip:]
            
            phen_state = phen_node.genotype[0, phen_ind]
            prev_phen_state = phen_node.up.genotype[0, phen_ind]
            
            subscore += np.abs((1.333 * prev_phen_state * prev_gene_state) + 
                              (.666 * prev_phen_state * gene_state) + 
                              (.666 * phen_state * prev_gene_state) + 
                              (1.333 * phen_state * gene_state) - 
                              phen_state - 
                              prev_phen_state - 
                              gene_state - 
                              prev_gene_state + 
                              1)
           
    if dist_only:
        hist_ = np.histogram(subscore, bins=int(1e7))
        fit_dist = rv_histogram(hist_)
        fit_dist.bin = np.diff(hist_[1]).max()
        return fit_dist
    if dist is not None:
        return dist.sf(subscore)
    else:
        return subscore

In [10]:
def GetPhyloEventScore(tree, phenotree, phen_ind, skip=0, with_rand=False):
    """
    Collects co-occuring events along the phylogeny
    
                                 /--(1,1)
                       /--(0,1)-|
             /--(0,1)-|          \--(0,1)
    --(0,1)-|          \--(1,1)
             \--(0,1)

    Would translate to the following contingency table:

    T\Q| 0  1
    ---------
     0 | 4  1
     1 | 1  0

    :param tree: ete3 tree
    :param phen_ind: index of target T
    :param skip: number of qualities to skip
    :return: A contingency table for each available quality in a nodes 'genotype' value
    """

    score = np.zeros(tree.genotype.shape[1] - skip)
    node_to_arr = lambda n: np.array(n.genotype.todense().astype(np.int8))[0]
    contingency_sum = 100
    for i, (cur_node, phen_node) in tqdm.tqdm(enumerate(zip(tree.traverse(), phenotree.traverse())), 
                                         total=population+1, desc='Iterating tree'):
        if not cur_node.is_root():
            if not cur_node.is_leaf() and with_rand and cur_node.random[phen_ind]: continue
            node = node_to_arr(cur_node)
            prev_node = node_to_arr(cur_node.up)
            
            gene_state = node[skip:]
            prev_gene_state = prev_node[skip:]
            
            phen_state = phen_node.genotype[0, phen_ind]
            prev_phen_state = phen_node.up.genotype[0, phen_ind]
            
            phen_event = (prev_phen_state != phen_state)
            gene_event = ~np.equal(prev_gene_state, gene_state)
            score = 
            
            hist = 2 * phen_event + gene_event  # current state
            if with_rand: hist[cur_node.up.random[skip:]] = -1
            # mark non-deterministic transition as invalid
            if not cur_node.is_leaf() and with_rand: 
                hist[cur_node.random[skip:]] = -1
            hist_kmers.append(hist)
            
    bins = lambda x: np.bincount(x[x >= 0], minlength=4) # TODO mem inefficiant
    if contingencies is None:
        contingencies = np.apply_along_axis(bins, 0, hist_kmers)
    else:
        contingencies += np.apply_along_axis(bins, 0, hist_kmers) 
    return contingencies

SyntaxError: invalid syntax (<ipython-input-10-e7e67f0fd847>, line 42)

In [ ]:
def GetPhyloEventContingency(tree, phenotree, phen_ind, skip=0, with_rand=False):
    """
    Collects co-occuring events along the phylogeny
    
                                 /--(1,1)
                       /--(0,1)-|
             /--(0,1)-|          \--(0,1)
    --(0,1)-|          \--(1,1)
             \--(0,1)

    Would translate to the following contingency table:

    T\Q| 0  1
    ---------
     0 | 4  1
     1 | 1  0

    :param tree: ete3 tree
    :param phen_ind: index of target T
    :param skip: number of qualities to skip
    :return: A contingency table for each available quality in a nodes 'genotype' value
    """

    hist_kmers = []
    node_to_arr = lambda n: np.array(n.genotype.todense().astype(np.int8))[0]
    contingency_sum = 100
    contingencies = None
    for i, (cur_node, phen_node) in tqdm.tqdm(enumerate(zip(tree.traverse(), phenotree.traverse())), 
                                         total=population+1, desc='Iterating tree'):
        if (i+1) % contingency_sum == 0 :
            # mem inefficiant here
            bins = lambda x: np.bincount(x[x >= 0], minlength=4)
            if contingencies is None:
                contingencies = np.apply_along_axis(bins, 0, hist_kmers)
            else:
                contingencies += np.apply_along_axis(bins, 0, hist_kmers)
            hist_kmers = []
            
        if not cur_node.is_root():
            if not cur_node.is_leaf() and with_rand and cur_node.random[phen_ind]: continue
            node = node_to_arr(cur_node)
            prev_node = node_to_arr(cur_node.up)
            
            gene_state = node[skip:]
            prev_gene_state = prev_node[skip:]
            
            phen_state = phen_node.genotype[0, phen_ind]
            prev_phen_state = phen_node.up.genotype[0, phen_ind]
            
            phen_event = (prev_phen_state != phen_state)
            gene_event = ~np.equal(prev_gene_state, gene_state)
            
            hist = 2 * phen_event + gene_event  # current state
            if with_rand: hist[cur_node.up.random[skip:]] = -1
            # mark non-deterministic transition as invalid
            if not cur_node.is_leaf() and with_rand: 
                hist[cur_node.random[skip:]] = -1
            hist_kmers.append(hist)
            
    bins = lambda x: np.bincount(x[x >= 0], minlength=4) # TODO mem inefficiant
    if contingencies is None:
        contingencies = np.apply_along_axis(bins, 0, hist_kmers)
    else:
        contingencies += np.apply_along_axis(bins, 0, hist_kmers) 
    return contingencies

In [ ]:
from scipy.stats import rv_discrete
def SetHomoplasy(tree, skip=0):
    """
    Get distribution of homoplasy
    """
    order = tree.traverse()
    root = next(order)
    homoplasy_hist = sp.csr_matrix(np.zeros(root.genotype.shape[1]-1))
    for cur_node in tqdm.tqdm(order, total=population):
        homoplasy_hist += (cur_node.genotype[0, skip:] != cur_node.up.genotype[0, skip:])
    homoplasy_dist = pd.Series(homoplasy_hist.data).value_counts(normalize=True)
    fit_dist = rv_discrete(values=(homoplasy_dist.index.astype(int), homoplasy_dist.values.astype(float)))
    root.homoplasy = fit_dist
    return tree

In [ ]:
colnames = ['ods', 'contingency']
from scipy.stats import rv_histogram
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
pandas2ri.activate()
statspackage = importr('stats', robject_translations={'format_perc': '_format_perc'})

def tables_to_pval(tables, pval_type='fisher', twotail=False):
    No_Phen_no_Kmer, Has_Kmer_no_Phen, Has_Phen_no_Kmer, Has_Kmer_and_Phen = tables

    # if ant zero's add 0.5 to all cells
    append = ~((No_Phen_no_Kmer > 0) * (Has_Kmer_no_Phen > 0) * (Has_Kmer_and_Phen > 0) * (
                Has_Phen_no_Kmer > 0)) * 0.5
    Has_Kmer_and_Phen = Has_Kmer_and_Phen + append
    Has_Kmer_no_Phen = Has_Kmer_no_Phen + append
    Has_Phen_no_Kmer = Has_Phen_no_Kmer + append
    No_Phen_no_Kmer = No_Phen_no_Kmer + append

    log_ods = np.log(np.divide((Has_Kmer_and_Phen * No_Phen_no_Kmer), (Has_Kmer_no_Phen * Has_Phen_no_Kmer)))
    
    if pval_type == 'fisher':
        print('calculating pvals')
        fisher = np.vectorize(lambda x:  list(map(list, statspackage.fisher_test(x, alternative='g')[:3])))
        fisher_input = list(zip(zip(Has_Kmer_and_Phen, Has_Kmer_no_Phen), zip(Has_Phen_no_Kmer, No_Phen_no_Kmer)))
        fisher_unique, curr_inverse_ind = np.unique(fisher_input, return_inverse=True, axis=0) #condence to uniq
        res = pd.DataFrame(fisher(pd.Series(list(fisher_unique))).tolist())[0].apply(lambda x: x[0])
        res = res.iloc[curr_inverse_ind].values
    elif pval_type == 'hist':     
        print('calculating pvals')
        hist_ = np.histogram(log_ods, bins=int(1e7))
        fit_dist = rv_histogram(hist_)
        fit_dist.bin = np.diff(hist_[1]).max()
        if twotail:
            #res = fit_dist.sf(np.abs(log_ods), loc=fit_dist.bin) + fit_dist.cdf(-np.abs(log_ods), loc=-fit_dist.bin)
            res = fit_dist.sf(np.abs(log_ods)) + fit_dist.cdf(-np.abs(log_ods))
        else:    
            res = fit_dist.sf(log_ods)#, loc=fit_dist.bin)
    elif pval_type == 'hist_only':
        print('calculating distribution')
        hist_ = np.histogram(log_ods, bins=int(1e7))
        fit_dist = rv_histogram(hist_)
        fit_dist.bin = np.diff(hist_[1]).max()
        return fit_dist, pd.DataFrame([log_ods, tables.T], index=colnames).T
    elif type(pval_type) is rv_histogram:
        print('calculating pvals')
        if twotail:
            #res = pval_type.sf(np.abs(log_ods), loc=pval_type.bin) + pval_type.cdf(-np.abs(log_ods), loc=-pval_type.bin)
            res = pval_type.sf(np.abs(log_ods)) + pval_type.cdf(-np.abs(log_ods))
        else:
            res = pval_type.sf(log_ods)#, loc=pval_type.bin)
        pval_type = 'input_hist'
    return pd.DataFrame([res, log_ods, tables.T], index=['pval_'+pval_type]+colnames).T

In [ ]:
from scipy.stats import pearsonr, spearmanr
def ods_fig(tot_tables, ods_columns):
    plt.close('all')
    with plt.style.context('default'):
        for col in ods_columns:
            plt.plot(sorted(tot_tables[col]), np.arange(len(tot_tables)) / len(tot_tables), label=col)
        plt.xlim(-6,6)
        # plt.xlabel(str(spearmanr(tot_tables['ods'], tot_tables['ods_recon'])))
        plt.legend()
        plt.title(str(n_mutations)+' mutations'+(' JointLike' if max_like else ' MaxPars')+': ods ratio cdf')
        if save_fig: plt.savefig(str(n_mutations)+('_max_like' if max_like else '')+'_ods.png')
        plt.show()

In [ ]:
from scipy.stats import pearsonr, spearmanr
def ods_ratio_sf(tot_tables, ods_cols, outpath=''):
    plt.close('all')
    with plt.style.context('default'):
        for col in ods_cols:
            plt.plot(sorted(tot_tables[col], reverse=True), np.arange(len(tot_tables)) / len(tot_tables), label=col)
        plt.yscale('log')
        plt.xlabel(str(spearmanr(tot_tables[ods_cols[0]], tot_tables[ods_cols[1]])))
        plt.legend()
        plt.title(str(n_mutations)+' mutations'+(' JointLike' if max_like else ' MaxPars')+': ods ratio sf')

        if save_fig: plt.savefig(os.path.join(outpath, str(n_mutations)+('_max_like' if max_like else '')+'_ods.png'))
        plt.show()

In [ ]:
plt.close('all')
def pval_hists(tables, pval_cols, cmap='rainbow', outpath=''):
    colors = plt.cm.get_cmap(cmap)(range(0, 255, 256//len(pval_cols)))
    with plt.style.context('default'):
        ax = plt.subplot()
        for col, color in zip(pval_cols, colors):
            ax.loglog(sorted(tables[col]), np.arange(len(tables)) / len(tables), label=col, color=color)
        ax.loglog(np.arange(len(tables)) / len(tables), np.arange(len(tables)) / len(tables), label='diag')
        ax.set_xlabel(str(n_mutations) +' mutations'+(' JointLike' if max_like else ' MaxPars')+': pval cdf')
        ax.set_ylabel('population fraction')
        cell_text = spearmanr(tot_tables[pval_cols])[0]
        cell_text = np.vectorize(lambda x: "{:.2e}".format(x))(cell_text)
        labels = pval_cols
        the_table = ax.table(cellText=cell_text,
                              colColours=colors,
                              colLabels=labels,
                              loc='top')

        if save_fig: plt.savefig(os.path.join(outpath, str(n_mutations)+('_max_like' if max_like else '')+'_pval.png'))
        plt.show()

In [ ]:
def pval_corr(tot_tables, pval_cols, outpath=''):
    plt.close('all')
    cell_text = spearmanr(tot_tables[pval_cols])[0]
    cell_text = np.vectorize(lambda x: "{:.2e}".format(x))(cell_text)
    with plt.style.context('default'):
        tmp_pval = tot_tables[pval_cols][~(tot_tables[pval_cols] == 0).any(axis=1)]
        for i, col in enumerate(pval_cols[1:]):
            plt.scatter(np.log10(tmp_pval[pval_cols[0]].astype(float)), np.log10(tmp_pval[col].astype(float)),
                        alpha=.1, label=col+' pval: ' + cell_text[0][i+1])
        plt.xlabel('log pval '+pval_cols[0])
        plt.legend()
        plt.title('Fisher pval correlation'+(' JointLike' if max_like else ' MaxPars'))
        if save_fig: plt.savefig(os.path.join(outpath, str(n_mutations)+('_max_like' if max_like else '')+'_scatter.png'))
        plt.show()
        del tmp_pval

In [ ]:
def hist_to_spearman(table, pval_main, cols, hist, outpath=''):
    with plt.style.context('default'):
        plt.close('all')
        for col in cols:
            pval_hist = []
            for val in range(1,int(hist.max()+2)):
                tmp_pval = table.loc[(hist<val).indices]
                spear = spearmanr(tmp_pval[[pval_main, col]])[0]
                pval_hist.append(spear)
            plt.plot(range(int(hist.max()+1)), pval_hist, label=col)
        plt.xlabel('tree recon distance')
        plt.ylabel('spearman correlation for d or lower distance')
        plt.title('spearman for recon distance')
        plt.legend()
        plt.savefig(os.path.join(outpath, str(n_mutations)+('_max_like' if max_like else '')+'_spearman_distance.png'))
        plt.show()

        plt.close('all')
        hist = sorted(hist.toarray()[0], reverse=True)
        plt.plot(hist, np.arange(len(hist)) / len(hist), label='source')
        plt.yscale('log')
        plt.title('recon distance over population sf')
        plt.xlabel('recon distance')
        plt.ylabel('population with <= recon distance')
        plt.savefig(os.path.join(outpath, str(n_mutations)+('_max_like' if max_like else '')+'_distance_sur.png'))
        plt.show()

# Pipe

In [ ]:
"""
Plan:

GroundTruth
1) build a tree (currently constant branch length)
2) simulate genotypes with a (fixed or from poiss dist) number of events by subsampling branches of tree
3) select a first genotype as phenotype
4) calculate contingencies of co-occuring events with phenotype of all traits
5) set pval as sf function of the calculated histogram of ods

MyCalc
6) reconstruct event of the phylogeny
7) calculate contingencies and ods from reconstructed tree to GroundTruth phenotype
8) set custom pval as the fisher test

TreeWas
9) estimate homoplay from ancestral reconstruction (6)
10) simulate 10x times the number of events
11) calculate TreeWas scores from simulated set to GroundTruth phenotype (assumed to be perfectly reconstructed)
12) calculate TreeWas scores from reconstructed tree to GroundTruth phenotype
"""

In [ ]:
from scipy.stats import poisson
import os

In [ ]:
# Global (program) params
leaf_population = 1000
n_genes = int(5e4)
n_mutations = 100
homoplasy = poisson(n_mutations)
max_like = False # Maxpasimony is used over max like in TreeWas: appendix 2
tips = False
subs = False
para = True
use_homop = False
save_fig = True
outpath = 'figs'
if not os.path.exists(outpath): os.mkdir(outpath)

In [ ]:
# 1) build a tree
GroundTruthTree = ete3.Tree()
GroundTruthTree.populate(leaf_population)
for i,n in enumerate(tqdm.tqdm(GroundTruthTree.traverse())): n.id = i
population = i
m_rate = (n_mutations/population)

In [ ]:
# for branch prportional (0-1) use fro every mut: 
# just set a branch distribution
# np.random.choice(population, lims.max(), p=branches, replace=False)

In [ ]:
# 2) simulate a set number of events by subsampling branches of tree
# TreeWas has a transition matrix Q for (g,p) to get number of interesting genes. We do things a bit differant

# for branch prportional (0-1) use fro every mut: np.random.choice(population, lims.max(), p=branches, replace=False)
if use_homop:
    lims = homoplasy.rvs(size=n_genes)
    events_per_mut = (np.argsort(np.random.rand(n_genes, population-1))[:, :lims.max()] + 1)
    GroundTruthTree = RandomAtributes(GroundTruthTree, population, n_genes, events_per_mut, lims=lims)
else:
    events_per_mut = (np.argsort(np.random.rand(n_genes, population-1))[:, :n_mutations] + 1)
    GroundTruthTree = RandomAtributes(GroundTruthTree, population, n_genes, events_per_mut)

# 3) select a phenotype
# 4) calculate contingencies of co-occuring events with phenotype of all traits
GroundTruthTables = events_to_contingency(events_per_mut)
ground_truth_tips = GetPhyloTipsContingency(GroundTruthTree, GroundTruthTree, phen_ind=0, skip=1)
del events_per_mut

# 5) set pval as sf function of the calculated histogram of ods
GroundTruthTables_para = tables_to_pval(np.transpose(GroundTruthTables), 'hist')
ground_truth_tips = tables_to_pval(ground_truth_tips, 'hist', twotail=True)
GroundTruthTables = GroundTruthTables_para.join(ground_truth_tips, lsuffix='_para', rsuffix='_tips')
del GroundTruthTables_para, ground_truth_tips

In [ ]:
# 6) reconstruct event of the phylogeny
# 6.1) choose ancestral reconstruction method
if max_like:
    ReconTree = JointMaxLiklihood(None, GroundTruthTree.copy(), {}, do_tips=True, naming=False, m_rate=m_rate)
else:
    ReconTree = MaxParsimonyNoTable(None, GroundTruthTree.copy(), {}, do_tips=True)

In [ ]:
ReconHist = sp.csr_matrix(np.zeros(shape=(1, n_genes)))
for og_node, recon_node in zip(GroundTruthTree.traverse(), ReconTree.traverse()):
    ReconHist += (og_node.genotype != recon_node.genotype)
ReconHist = ReconHist[0, 1:] # no pheno

In [ ]:
# 7) calculate contingencies and ods from reconstructed tree to GroundTruth phenotype
tables_recon_para = GetPhyloEventContingency(ReconTree, GroundTruthTree, phen_ind=0, skip=1, with_rand=True)
tables_recon_tips = GetPhyloTipsContingency(ReconTree, GroundTruthTree, phen_ind=0, skip=1)
tables_recon_subs = GetPhyloSebsequentScore(ReconTree, GroundTruthTree, phen_ind=0, skip=1, with_rand=False) 
# 8) set custom pval as the fisher test
tables_recon = tables_to_pval(tables_recon_para, 'fisher')

In [ ]:
# 9) estimate homoplay from ancestral reconstruction (6) (optional - can be constant)
ReconTree = SetHomoplasy(ReconTree, skip=1)

# TODO save space no need for GroundTruthTree genotypes from here

In [ ]:
# 10) simulate 10x times the number of events
# TODO could save space by calculatin 12 while building tree, then erasing prev data
n_sim_genes = min(n_genes*10, int(5e5))
sim_n_mutations_max = ReconTree.homoplasy.xk.max()
chunks = int(1e5)
parts = int(n_sim_genes//chunks)
sim_events_per_mut = []
for i in tqdm.tqdm(range(parts), total=parts, desc='simulating events in chucks'):
    sim_events_per_mut.append((np.argsort(np.random.rand(chunks, population-1))[:, :sim_n_mutations_max] + 1))
sim_events_per_mut = np.concatenate(sim_events_per_mut)
lims = ReconTree.homoplasy.rvs(size=n_sim_genes)
SimTree = RandomAtributes(GroundTruthTree.copy(), population, n_sim_genes, sim_events_per_mut, lims=lims)

# save space no need for ReconTree
del ReconTree
del sim_events_per_mut

In [ ]:
# 11) calculate TreeWas scores distribution from simulated set to GroundTruth phenotype
# 12) calculate TreeWas scores from reconstructed tree to GroundTruth phenotype
if tips:
    tables_sim_raw = GetPhyloTipsContingency(SimTree, GroundTruthTree, phen_ind=0, skip=1)
    score_1_dist, tip_sim = tables_to_pval(tables_sim_raw, 'hist_only')
    tables_sim = tables_to_pval(tables_recon_tips, score_1_dist, twotail=True)[['pval_input_hist']]
    tables_sim.columns = ['tips_score']
    del tables_recon_tips

if para:
    tables_sim_raw = GetPhyloEventContingency(SimTree, GroundTruthTree, phen_ind=0, skip=1, with_rand=False) 
    score_2_dist, para_sim = tables_to_pval(tables_sim_raw, 'hist_only')
    tables_sim = tables_to_pval(tables_recon_para, score_2_dist)[['pval_input_hist']]
    tables_sim.columns = ['parallel_score']
    del tables_recon_para, tables_sim_raw

if subs:
    score_3_dist = GetPhyloSebsequentScore(SimTree, GroundTruthTree, phen_ind=0, skip=1, with_rand=False, dist_only=True) 
    tables_sim = pd.DataFrame([score_3_dist.sf(tables_recon_subs)], columns='subs_score')

In [ ]:
tot_tables = GroundTruthTables.join(tables_recon, rsuffix='_recon').join(tables_sim, rsuffix='_treewas').dropna()

In [ ]:
# pval_cols = ['pval_hist_para', 'pval_hist_tips', 'pval_fisher', 'tips_score', 'parallel_score', 'subs_score']
pval_cols = ['pval_hist_para', 'pval_fisher', 'parallel_score']

In [ ]:
pval_hists(tot_tables, pval_cols, outpath=outpath)
# TODO fix hists

In [ ]:
pval_corr(tot_tables, pval_cols, outpath=outpath)

In [ ]:
hist_to_spearman(tot_tables, 'pval_hist_para', ['pval_fisher', 'parallel_score'], ReconHist, outpath=outpath)

In [ ]:
ods_ratio_sf(tot_tables, ['ods', 'ods_para'], outpath=outpath)